# Imports


In [ ]:
options = {}

# Set the path to the data file
# eg. options["data"] = "data.csv" if the file is in the same directory as this notebook
options["data"] = ""

# Uncomment the following lines if you are using Google Colab
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# %cd "drive/MyDrive/PATH/TO/HERE"

# Use the following line to install any missing packages
# %pip install [packages]

import chardet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import xgboost as xgb
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split,
    cross_val_score,
)
from sklearn.preprocessing import (
    OrdinalEncoder,
    PowerTransformer,
    StandardScaler,
    PolynomialFeatures,
)
from sklearn.utils import resample
from tqdm.notebook import tqdm, trange


# Data Collection


In [ ]:
# Read in the data
with open(options["data"], "rb") as rawdata:
    encoding = chardet.detect(rawdata.read(100000))
df = pd.read_csv(options["data"], encoding=encoding["encoding"])
print(df)


# Options


Set the variable you want to predict here, as well as any ordinal columns. Ordinal columns are columns that have a natural ordering, such as "low", "medium", "high".


In [ ]:
options["target"] = ""
options["ordinal"] = []


# Cleaning and Preprocessing


We begin by performing a series of data preprocessing steps to prepare the dataset for machine learning modeling.

First, we clean the data by removing non-numeric characters from numerical columns and imputing any missing values, and encoding categorical variables, treating ordinal and nominal categories differently. Then we separates the dataset into features and the target variable, and remove any outliers using the Isolation Forest method. Next, we transform the numerical features to make their distribution more Gaussian-like (using the Yeo-Johnson method), which helps improve the performance of many machine learning algorithms. Finally, we standardize numerical features to ensure they are on the same scale. This is particularly important for certain machine learning models that are sensitive to the scale of the input features.


In [ ]:
# Remove commas in numerical columns
for col in df.columns:
    if df[col].dtype == "object":
        try:
            df[col] = pd.to_numeric(df[col].str.replace(",", ""))
        except ValueError:
            pass

# Impute missing values - Mean for numerical columns
for col in df.select_dtypes(include=["int", "float"]):
    df[col] = df[col].fillna(df[col].mean())

# Impute missing values - Mode for categorical columns
for col in df.select_dtypes(include=["object"]):
    df[col] = df[col].fillna(df[col].mode()[0])

# Ordinal encoding for ordinal features
for feature in options["ordinal"]:
    if feature in df.columns:
        ordinal_encoder = OrdinalEncoder()
        df[feature] = ordinal_encoder.fit_transform(df[[feature]])

# One-hot encoding for categorical columns
df = pd.get_dummies(df)

# Split the data into target and features
y = df[options["target"]]
X = df.drop(options["target"], axis=1)

# Detect and remove outliers using Isolation Forest on X
iso = IsolationForest()
yhat = iso.fit_predict(X)
mask = yhat != -1
X = X[mask]
y = y[mask]  # It's important to apply the same filter to y

# Apply power transformation to make data more Gaussian-like and handle skewness
pt = PowerTransformer(method="yeo-johnson", standardize=True)
X[X.select_dtypes(include=["int", "float"]).columns] = pt.fit_transform(
    X.select_dtypes(include=["int", "float"])
)

# Standard scaling for numerical features
scaler = StandardScaler()
X[X.select_dtypes(include=["int", "float"]).columns] = scaler.fit_transform(
    X.select_dtypes(include=["int", "float"])
)


# Feature Engineering


Here we use polynomial transformations to create new features from the original data set. We test different degrees of polynomial features to identify the degree that produces the best model performance. The process of creating and selecting the right polynomial features helps to capture more complex relationships in the data that might be missed by a linear model.

For each degree of polynomial features, we train an XGBoost regression model using bootstrap samples of the data and evaluates their performance using cross-validation. This ensures the robustness of the model's performance estimate and helps avoid overfitting to the training data.


In [ ]:
size_threshold = 5000
tree_method = "gpu_hist" if torch.cuda.is_available() else "exact"

# Define a range of degrees to try
degrees = range(1, 5)

best_degree = None
best_score = float("-inf")
best_X_poly = None
best_feature_names = None

# Iterate over degrees and evaluate performance with cross-validation
print("Generating polynomial features...")
for degree in tqdm(degrees):
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False, interaction_only=False)
    X_poly = poly.fit_transform(X)

    # Generate feature names manually
    feature_names = []
    for power in poly.powers_:
        name = []
        for feature, p in zip(X.columns, power):
            if p != 0:
                if p == 1:
                    name.append(feature)
                else:
                    name.append(f"{feature}^{p}")
        feature_names.append(" ".join(name))

    if X_poly.shape[0] < size_threshold:  # If the dataset is small, bootstrap
        n_bootstrap = 100
        bootstrap_models = []
        print("Bootstrapping for degree", degree)
        for i in trange(n_bootstrap):
            # Create a bootstrap sample from the original dataset
            X_resampled, y_resampled = resample(X_poly, y, replace=True)

            # Train model on bootstrap sample
            model = xgb.XGBRegressor(tree_method=tree_method, random_state=42)
            model.fit(X_resampled, y_resampled)

            # Add the model to the list of bootstrap models
            bootstrap_models.append(model)

        # Evaluate with cross-validation: Average the scores from all models
        scores = [
            cross_val_score(
                model, X_poly, y, cv=5, scoring="neg_mean_squared_error"
            ).mean()
            for model in tqdm(bootstrap_models)
        ]
        mean_score = np.mean(scores)
    else:
        model = xgb.XGBRegressor(tree_method=tree_method, random_state=42)
        model.fit(X_poly, y)
        mean_score = cross_val_score(
            model, X_poly, y, cv=5, scoring="neg_mean_squared_error"
        ).mean()

    # If this degree gives a better score, update the best values
    if mean_score > best_score:
        best_degree = degree
        best_score = mean_score
        best_X_poly = X_poly
        best_feature_names = feature_names

print(f"Best degree: {best_degree}, Cross-validation score: {best_score}")
best_X_poly_df = pd.DataFrame(best_X_poly, columns=best_feature_names)


# Hyperparameter Tuning and Feature Selection


Now we train an XGBoost regression model to predict the target variable. To improve the model's performance, hyperparameter tuning and feature selection are performed.

RandomizedSearchCV performs a randomized search over the hyperparameters, trying out a fixed number of hyperparameter settings sampled from the specified distributions. The number of settings that are tried is given by n_iter. For each setting, the model's performance is evaluated using 4-fold cross-validation. This means the training data is split into 4 parts, the model is trained on 3 parts and evaluated on the 4th part. This process is repeated 4 times, each time with a different part used for evaluation. The average score over the 4 evaluations is taken as the score for that setting. The setting that gives the highest score is selected as the best hyperparameters.

The forward feature selection strategy then starts by including the single feature that yields the highest cross-validated score and continues adding features one by one until the inclusion of additional features does not yield a significant improvement in the cross-validated score. This way, the most important features that contribute to predicting Service Appointments are selected.


In [ ]:
# Split the data into training, validation and testing sets
X_train, X_val, y_train, y_val = train_test_split(best_X_poly_df, y, random_state=42)

# Define a range of values for each hyperparameter
hyperparameters = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [2, 3, 4, 5, 6],
    "learning_rate": [0.01, 0.1, 0.2, 0.3],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "min_child_weight": [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
    "gamma": [0, 0.25, 0.5, 1.0],
    "reg_lambda": [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
    "reg_alpha": [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
}

# Random search with 4-fold cross validation
random_cv = RandomizedSearchCV(
    xgb.XGBRegressor(tree_method=tree_method, random_state=42),
    hyperparameters,
    n_iter=50,
    cv=4,
    n_jobs=-1,
    verbose=1,
    scoring="neg_mean_absolute_error",
    return_train_score=True,
    random_state=42,
)
random_cv.fit(X_train, y_train)
print("Tuned XGBoost Parameters: {}".format(random_cv.best_params_))
print("Best score is {}".format(random_cv.best_score_))

# Train a model with the optimal hyperparameters
model_full = xgb.XGBRegressor(
    **random_cv.best_params_, tree_method=tree_method, random_state=42
)
model_full.fit(X_train, y_train)

# Get feature importances
importances = model_full.feature_importances_
feature_importances = pd.Series(importances, index=best_feature_names).sort_values(
    ascending=False
)

# List to store cross-validation scores
cv_scores = []

# Iterate over numbers of features
print("Calculating optimal number of features...")
for n_features in trange(1, len(feature_importances) // 100):
    # Select top n features
    selected_features = feature_importances[:n_features].index

    # Select these top features from the original dataset
    X_train_selected = X_train[selected_features]
    X_val_selected = X_val[selected_features]

    # Compute cross-validation score
    score = cross_val_score(model, X_train_selected, y_train, cv=5).mean()

    # Append the score to the list of scores
    cv_scores.append(score)

# Convert list of scores to a series
cv_scores = pd.Series(cv_scores, index=range(1, len(feature_importances) // 100))

# Select the top N features that give the maximum score
optimal_n_features = cv_scores.idxmax()
selected_features = feature_importances[:optimal_n_features].index
print(feature_importances[selected_features])

# Plot the feature importances
plt.figure(figsize=(10, 6))
feature_importances[selected_features].plot(kind="bar", color="skyblue")
plt.title(f"Top {len(selected_features)} Important Features")
plt.ylabel("Importance")
plt.xlabel("Features")
plt.show()

# Select these top features from the original dataset
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]

# Train a reduced model with the optimal hyperparameters
model = xgb.XGBRegressor(
    **random_cv.best_params_, tree_method=tree_method, random_state=42
)
model.fit(X_train_selected, y_train)


# Model Evaluation


Finally, we evaluate and compare the performance of the two machine learning models. The 'Full Model' is trained using all the features in the dataset, while the 'Reduced Model' is trained using a subset of the most important features.

The evaluation is done on a separate validation dataset and the performance metrics used for comparison are Mean Absolute Error (MAE), Root Mean Square Error (RMSE), and the Coefficient of Determination (R2 Score). MAE is the average absolute difference between the predicted and actual values. This gives an idea of how wrong the predictions were. RMSE is the square root of the average of squared differences between prediction and actual observation. This metric penalizes larger errors more than the MAE. R2 is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model. It provides a measure of how well future samples are likely to be predicted by the model.

These metrics provide a quantifiable measure of the models' prediction errors and their ability to explain the variance in the target variable. The lower the MAE and RMSE, the better the model. The higher the R2 score, the better the model.


In [ ]:
models_and_data_sets = {
    "Full Model": {"model": model_full, "Validation": X_val},
    "Reduced Model": {"model": model, "Validation": X_val_selected},
}

# Iterate over models and their corresponding data sets:
for model_name, model_dict in models_and_data_sets.items():
    # Make predictions
    y_pred = model_dict["model"].predict(model_dict["Validation"])

    # Compute metrics
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    r2 = model_dict["model"].score(model_dict["Validation"], y_val)
    print(f"{model_name}: MAE = {mae}, RMSE = {rmse}, R2 = {r2}")
